# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas as hv 
import pandas as pd
import geopy as gpy
import matplotlib
import requests





# Import API key
from api_keys import geoapify_key

ModuleNotFoundError: No module named 'api_keys'

In [303]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
1,1,farsund,58.0948,6.8047,13.30,100,0,7.65,NO,1666108228
2,2,new norfolk,-42.7826,147.0587,11.72,58,12,1.34,AU,1666108230
3,3,jamestown,42.0970,-79.2353,5.77,77,100,9.77,US,1666107934
4,4,lanzhou,36.0564,103.7922,14.53,48,59,1.20,CN,1666108230


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [319]:
%%capture --no-display

# Create the initial scatter plot with colorful city dots and a background map
map_plot = city_data_df.hvplot.points(
    x='Lng', y='Lat', size='Humidity', color='Humidity',
    cmap='viridis', hover_cols=['City'], alpha=0.5,
    tiles='OSM',  # Specify the background map tile source
    geo=True,  # Add the geo parameter
    scale=1,  # Set the scale parameter to control the size of the points
    width=800, height=500
)

map_plot.opts(bgcolor='skyblue')

map_plot




:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,City)

In [ ]:
# Create a leaflet map 

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [312]:
# Narrow down cities that fit criteria and drop any results with null values
# Filter the data based on 'Max Temp' range and drop null values
# Filter the DataFrame for cities with temperature between 20 and 30 degrees Celsius
ideal_weather_df = city_data_df[(city_data_df['Max Temp'] > 20) & (city_data_df['Max Temp'] < 30)]

# Drop any null values in the temperature column
ideal_weather_df = ideal_weather_df.dropna(subset=['Max Temp'])

# Create the scatter plot with the filtered DataFrame
map_plot = ideal_weather_df.hvplot.points(
    x='Lng', y='Lat', size='Humidity', color='Humidity',
    cmap='viridis', hover_cols=['City'], alpha=0.5,
    tiles='OSM', geo=True, scale=1,
    width=800, height=500
)

map_plot.opts(bgcolor='skyblue')
map_plot

# Drop any rows with null values
# YOUR CODE HERE

# Display sample data
# YOUR CODE HERE

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,City)

### Step 3: Create a new DataFrame called `hotel_df`.

In [313]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# Create a new DataFrame 'hotel_df' with selected columns from 'city_data_df'
hotel_df = city_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,faya,SA,18.3851,42.4509,35,
1,farsund,NO,58.0948,6.8047,100,
2,new norfolk,AU,-42.7826,147.0587,58,
3,jamestown,US,42.0970,-79.2353,77,
4,lanzhou,CN,36.0564,103.7922,48,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [ ]:
# Set parameters to search for a hotel
radius = 50000
params = {
    'radius': radius
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from the DataFrame
    lat_value = row['Lat']
    long_value = row['Lng']

    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params['filter'] = f"distance(lat,lng,{long_value},{lat_value})<5000"
    params['bias'] = f"distance(lat,lng,{lat_value},{long_value})"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    api_data = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_name = api_data["features"][0]["properties"]["name"]
        hotel_df.loc[index, "Hotel Name"] = hotel_name
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{row['City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data from hotel_df
print(hotel_df)

Starting hotel search
faya - nearest hotel: No hotel found
farsund - nearest hotel: No hotel found
new norfolk - nearest hotel: No hotel found
jamestown - nearest hotel: No hotel found
lanzhou - nearest hotel: No hotel found
ushuaia - nearest hotel: No hotel found
albany - nearest hotel: No hotel found
carnarvon - nearest hotel: No hotel found
sisimiut - nearest hotel: No hotel found
saint-pierre - nearest hotel: No hotel found
colorado - nearest hotel: No hotel found
nikolskoye - nearest hotel: No hotel found
trairi - nearest hotel: No hotel found
dobryanka - nearest hotel: No hotel found
qaanaaq - nearest hotel: No hotel found
mataura - nearest hotel: No hotel found
khatanga - nearest hotel: No hotel found
tasiilaq - nearest hotel: No hotel found
tiksi - nearest hotel: No hotel found
puerto ayora - nearest hotel: No hotel found
avarua - nearest hotel: No hotel found
nuevo laredo - nearest hotel: No hotel found
guiratinga - nearest hotel: No hotel found
rikitea - nearest hotel: No hot

KeyboardInterrupt: 

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [314]:
%%capture --no-display

# Update the hover_cols parameter to include 'City', 'Hotel Name', and 'Country'
map_plot = city_data_df.hvplot.points(
    x='Lng', y='Lat', size='Humidity', color='Humidity',
    cmap='viridis', hover_cols=['City', 'Hotel Name', 'Country'], alpha=0.5,
    tiles='OSM', geo=True, scale=1,
    width=800, height=500
)

map_plot.opts(bgcolor='skyblue')
map_plot

# Display the map
# YOUR CODE HERE

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,City,Country)